# BERT IRQA 기반의 챗봇 실습

* [CLS] token을 이용해서 Top-n개를 추출
* Top-n개를 하나씩 확인하며 paraphrase detection을 함
* paraphrase detection을 하여 question과 나의 query가 유사한 것이 확인되면 A 답변을 return

* 심리 상담 관련 챗봇 데이터 사용

* 데이터 다운로드 및 확인

In [ ]:
!git clone https://github.com/songys/Chatbot_data.git

In [ ]:
import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

* single term
  * input과 output이 하나로만 구성되어있는 QA dataset

In [ ]:
data.head()
'''
	                        Q	                 A	label
0	                 12시 땡!	  하루가 또 가네요.	    0
1	      1지망 학교 떨어졌어	   위로해 드립니다.	    0
2	     3박4일 놀러가고 싶다	여행은 언제나 좋죠.	    0
3	3박4일 정도 놀러가고 싶다	여행은 언제나 좋죠.	    0
4	               PPL 심하네	 눈살이 찌푸려지죠.	    0
'''

## Top-n 추출 모듈

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to(device)

* 데이터 확인
  * `chatbot_Question`과 `chatbot_Answer`의 index가 동일한 문장이 서로 pair됨

In [ ]:
chatbot_Question = data['Q'].values
chatbot_Answer = data['A'].values
print(chatbot_Question[0:3])
# ['12시 땡!' '1지망 학교 떨어졌어' '3박4일 놀러가고 싶다']
print(chatbot_Answer[0:3])
# ['하루가 또 가네요.' '위로해 드립니다.' '여행은 언제나 좋죠.']

### [CLS] token을 얻기 위한 함수

In [ ]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=32
    ).to(device)
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

* 사전에 정의된 모든 sentence에 대한 vector를 추출

In [ ]:
chatbot_Question_vectors = {}
for i, question in enumerate(chatbot_Question):
    chatbot_Question_vectors[i] = get_cls_token(question)

* 입력 문장 query와 가장 유사한 top-n개 추출

In [ ]:
import numpy as np

* cosine similarity를 구하는 함수
  * sklearn의 cosine similarity도 사용 가능함

In [ ]:
def custom_cosine_similarity(a,b):
    numerator = np.dot(a,b.T)
    a_norm = np.sqrt(np.sum(a * a))
    b_norm = np.sqrt(np.sum(b * b, axis=-1))

    denominator = a_norm * b_norm
    return numerator/denominator

In [ ]:
def return_top_n_idx(question, n):
    question_vector = get_cls_token(question)
    sentence_similarity = {}
    for i in chatbot_Question_vectors.keys():
        ir_vector = chatbot_Question_vectors[i]
        similarity = custom_cosine_similarity(question_vector, ir_vector)
        sentence_similarity[i] = similarity
    
    sorted_sim = sorted(sentence_similarity.items(), key=lambda x: x[1], reverse=True)
    return sorted_sim[0:n]

In [ ]:
print(return_top_n_idx("오늘 너무 힘들어", 5))  # top 5개 question id를 반환합니다.
# [(3285, array([[0.97600377]], dtype=float32)), (7121, array([[0.9664848]], dtype=float32)), (5947, array([[0.9598295]], dtype=float32)), (5959, array([[0.95737875]], dtype=float32)), (7176, array([[0.9529198]], dtype=float32))]

* Top-n의 결과

In [ ]:
print('most similar questions')
for result in return_top_n_idx("오늘 너무 힘들어", 5):
    print(chatbot_Question[result[0]])
'''
most similar questions
오늘 너무 힘들다
오늘 너무 힘드네
너무 힘들어
너무나도 힘들어
오늘따라 너무 힘드네
'''

print('\nmost similar answers')
for result in return_top_n_idx("오늘 너무 힘들어", 5):
    print(chatbot_Answer[result[0]])
'''
most similar answers
고생 많았어요.
오늘은 힘내려 하지 말아요. 저에게 기대세요.
지금 무슨 말을 해도 와닿지 않겠지만 잘할 수 있을 거예요.
억지로라도 긍정적인 생각을 해보세요.
힘든 날이네요.
'''

* Top-n의 결과
  * Top-1의 결과가 정답이 아닌 경우도 있음

In [ ]:
print('most similar questions')
for result in return_top_n_idx("너 이름이 뭐야?", 5):
    print(chatbot_Question[result[0]])
'''
most similar questions
우정이 뭐야?
너 뭐니?
할 줄 아는거 뭐야?
사랑의 끝이 뭐야?
너 누구?
'''

print('\nmost similar answers')
for result in return_top_n_idx("너 이름이 뭐야?", 5):
    print(chatbot_Answer[result[0]])
'''
most similar answers
힘들 때 같이 있는 거요.
저는 위로봇입니다.
당신의 삶을 응원해 드릴 수 있어요라고 감히 말해 봅니다.
사랑하지 않는 것이죠.
저는 마음을 이어주는 위로봇입니다.
'''

## 이진 분류 모델

* Top-n의 결과를 이진 분류 모델을 태워서 실제로 유사한지 검사함

* colab과 나의 google drive를 연결

In [ ]:
# 왼쪽 navi에서 폴더icon을 클릭해서 코드 가져올 수 있음
from google.colab import drive
drive.mount('/content/drive')

* 저장된 model을 불러오기

In [ ]:
from transformers import BertForSequenceClassification

MODEL_NAME = "/content/drive/MyDrive/P_KLUE/baseline_model/bert_two_sent_classifier"
classifier_model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
classifier_model.to(device)

In [ ]:
# predict함수
# 0: "non_similar", 1: "similar"
def sentences_predict(sent_A, sent_B):
    classifier_model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            sent_B,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=64
    )
    
    tokenized_sent.to('cuda:0')
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = classifier_model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)

    # if result == 0:
    #   result = 'non_similar'
    # elif result == 1:
    #   result = 'similar'
    return result

In [ ]:
print(sentences_predict('오늘 날씨가 어때요?','오늘의 날씨를 알려줘')) # similar
print(sentences_predict('오늘 날씨가 어때요?','기분 진짜 안좋다.')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','오늘 기분 어떠세요?')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','오늘 기분이 어때요?')) # non_similar
print(sentences_predict('오늘 날씨가 어때요?','지금 날씨가 어때요?')) # non_similar
print(sentences_predict('무협 소설 추천해주세요.','무협 장르의 소설 추천 부탁드립니다.')) # similar
print(sentences_predict('무협 소설 추천해주세요.','판타지 소설 추천해주세요.')) # non_similar
print(sentences_predict('무협 소설 추천해주세요.','무협 느낌나는 소설 하나 추천해주실 수 있으실까요?')) # similar
print(sentences_predict('메난민이 뭐야','너 메난민이지?')) # similar

* pipeline 합치기
  * 두 모듈을 합치기

* `return_top_n_idx(question, n)` : input question과 가장 유사한 question n개를 return

In [ ]:
def get_answer(question, n):
    results = return_top_n_idx(question, n) # top n개를 list로 받고
    for result in results:  # n개를 반복문을 돌면서
        ir_answer = chatbot_Answer[result[0]]
        ir_question = chatbot_Question[result[0]]
        if sentences_predict(question, ir_question) == 1:   # 이진분류 모델이 query<->question의 의미가 서로 같다고 판단되면?
            return ir_answer    # 정답을 반환합니다.
    return chatbot_Answer[results[0][0]] # 정답이 없는 경우 top-1이 어떤 답변이든 반환함
    # return "잘 모르겠어요."


In [ ]:
print(get_answer("너 이름이 뭐야?", 5))
# 저는 위로봇입니다.

In [ ]:
print(get_answer("나 지금 너무 우울해", 5))
# 꼼꼼한 거예요.

In [ ]:
print(get_answer("오늘 기분 어때?", 5))
# 숨 쉴만 했으면 좋겠네요.

In [ ]:
print(get_answer("바쁜가보네?", 5))
# 직접 확인해보세요.

In [ ]:
print(get_answer("어떻게 확인하는데?", 5))
# 잠시 차분하게 생각해봐요.

* 챗봇 구성
  * IRQA로 사전에 정의된 답변으로 답할 수 있는 경우 답함
  * 사전에 정의된 답변으로 답하지 못하는 경우 생성 모델을 통해 답변을 만들어냄